In [1]:
# Import statements
# Declare Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
import os
import pickle
from textwrap import wrap
import operator
import au_census_analysis_functions as cnss_func

# Set a variable for current notebook's path for various loading/saving mechanisms
nb_path = os.getcwd()
td_path = os.path.dirname(os.path.realpath(nb_path))

In [2]:
import table_funcs as tbl_func

In [13]:
import importlib
importlib.reload(cnss_func)

<module 'au_census_analysis_functions' from 'C:\\Users\\kempbri\\OneDrive - Mars Inc\\Documents\\GitHub\\Census-ASGS-Level-and-Model-Sensitivity\\au_census_analysis_functions.py'>

### Custom Functions
Below creates a number of custom functions. These are also replicated in the au_census_analysis_functions.py file but for the time being I am leaving them here if I need to re-run some of the cells executed prior to the implementation of these as imported functions.

In [3]:
# Show tables which have the above fields of information available
df_meta = pd.read_csv('{}\Data\Metadata\Metadata_2016_refined.csv'.format(td_path))
df_m_refined = df_meta[(df_meta['Age'] > 0) |
                      (df_meta['Income'] > 0) |
                      (df_meta['Occupation'] > 0) |
                      (df_meta['Family Composition'] > 0) |
                      (df_meta['Internet status'] > 0) |
                      (df_meta['Building occupation status'] > 0) |
                      (df_meta['Number of Bedrooms'] > 0) |
                      (df_meta['Place of Birth'] > 0) |
                      (df_meta['Country of Birth of Parents'] > 0) |
                      (df_meta['Year of Arrival'] > 0)]
tables_to_load = df_m_refined['Profile table'].unique().tolist()
load_features = ['Age','Income']

## Begin importing and exploring data

In [23]:
options_df = df_m_refined[['DataPack file','Table name']]
options_df = options_df.drop_duplicates(subset='DataPack file')
options_df = options_df.set_index('DataPack file')
options_df.head()

,Table name
DataPack file,
G01,Selected Person Characteristics by Sex
G03,Place of Usual Residence on Census Night by Age
G04A,Age by Sex
G04B,Age by Sex
G05,Registered Marital Status by Age by Sex


In [24]:
temp_df = pd.DataFrame(options_df['Table name'].value_counts()).reset_index()
temp_df.columns = ['Table name','Count']
options_df = options_df.reset_index().merge(temp_df, on='Table name')

In [27]:
options_df.loc[options_df.Count > 1, 'DataPack file'] = options_df['DataPack file'].str[:3]
options_df.drop_duplicates()

,DataPack file,Table name,Count
0,G01,Selected Person Characteristics by Sex,1
1,G03,Place of Usual Residence on Census Night by Age,1
2,G04,Age by Sex,2
4,G05,Registered Marital Status by Age by Sex,1
5,G06,Social Marital Status by Age by Sex,1
6,G07,Indigenous Status by Age by Sex,1
7,G09,Country of Birth of Person by Age by Sex,8
15,G10,Country of Birth of Person by Year of Arrival ...,3
18,G11,Proficiency in Spoken English/Language by Year...,4
22,G12,Proficiency in Spoken English/Language of Pare...,2


In [13]:
options_dict = [{'label': index, 'value': val} for index, val in options_df['Table name'].iteritems()]

options_dict

[{'label': 'G01', 'value': 'Selected Person Characteristics by Sex'},
 {'label': 'G03', 'value': 'Place of Usual Residence on Census Night by Age'},
 {'label': 'G04A', 'value': 'Age by Sex'},
 {'label': 'G04B', 'value': 'Age by Sex'},
 {'label': 'G05', 'value': 'Registered Marital Status by Age by Sex'},
 {'label': 'G06', 'value': 'Social Marital Status by Age by Sex'},
 {'label': 'G07', 'value': 'Indigenous Status by Age by Sex '},
 {'label': 'G09A', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09B', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09C', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09D', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09E', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09F', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09G', 'value': 'Country of Birth of Person by Age by Sex'},
 {'label': 'G09H', 'value': 'Country of Birth of Per

In [12]:
print ([x for x in options_df['Table name'].iteritems()])

[('G01', 'Selected Person Characteristics by Sex'), ('G03', 'Place of Usual Residence on Census Night by Age'), ('G04A', 'Age by Sex'), ('G04B', 'Age by Sex'), ('G05', 'Registered Marital Status by Age by Sex'), ('G06', 'Social Marital Status by Age by Sex'), ('G07', 'Indigenous Status by Age by Sex '), ('G09A', 'Country of Birth of Person by Age by Sex'), ('G09B', 'Country of Birth of Person by Age by Sex'), ('G09C', 'Country of Birth of Person by Age by Sex'), ('G09D', 'Country of Birth of Person by Age by Sex'), ('G09E', 'Country of Birth of Person by Age by Sex'), ('G09F', 'Country of Birth of Person by Age by Sex'), ('G09G', 'Country of Birth of Person by Age by Sex'), ('G09H', 'Country of Birth of Person by Age by Sex'), ('G10A', 'Country of Birth of Person by Year of Arrival in Australia'), ('G10B', 'Country of Birth of Person by Year of Arrival in Australia'), ('G10C', 'Country of Birth of Person by Year of Arrival in Australia'), ('G11A', 'Proficiency in Spoken English/Languag